<a href="https://colab.research.google.com/github/lsh4205/Traffic_Simulation/blob/main/traffic_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
print("Python version:", sys.version)

Python version: 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]


# Define logical 1-D domains #

The following class is a small wrapper around a 1-D Numpy array of values that define a discretized domain with a uniform step size.

In [2]:
class Domain1D:
    def __init__(self, step, min_val=-1, max_val=1):
        from numpy import linspace
        self.step = step
        n = int((max_val - min_val) / step) + 1
        self.vals = linspace(min_val, max_val, n)
        
    def __len__(self): # length in terms of # of grid points
        return len(self.vals)
    
    def __getitem__(self, I): # values by an arbitrary Numpy index-slice `I`
        return self.vals[I]

**Define a discrete time domain: $t \in [0, 120]$ with step size 0.1.**

In [3]:
T = Domain1D(0.1, min_val=0.0, max_val=120.0)
T[:]

array([0.000e+00, 1.000e-01, 2.000e-01, ..., 1.198e+02, 1.199e+02,
       1.200e+02])

**Define a discrete spatial domain: $x \in [-5, 5]$ with step size 0.01\**.

In [4]:
X = Domain1D(0.01, min_val=-5.0, max_val=5.0)
X[:]

array([-5.  , -4.99, -4.98, ...,  4.98,  4.99,  5.  ])